In [8]:
# !pip install openai
# !pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.2 MB/s eta 0:00:00


In [9]:
import openai
import pandas as pd
import numpy as np
import tiktoken

In [87]:
f = open("/content/sample_data/OPENAI_API_KEY.txt", "r")
openai_api_key = f.read()
# print(openai_api_key)
client = openai.OpenAI(api_key=openai_api_key)

In [5]:
llm_model = "gpt-3.5-turbo"

In [72]:
def get_completion_and_token_count(prompt,
                                   model=llm_model,
                                   temperature=0,
                                   max_tokens=500):
    messages = [{"role": "user", "content": prompt+' output in JSON format'}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,        # messages = [{"role": "user", "content": prompt}], a list of messages comprising the conversation so far.
        temperature=temperature,
        max_tokens=max_tokens,    # The maximum number of tokens that can be 'generated' by the chat completion.
    )

    content = response.choices[0].message.content

    token_dict = {
        'prompt_tokens':response.usage.prompt_tokens, # no. of input tokens counted by OpenAI API
        'completion_tokens':response.usage.completion_tokens,
        'total_tokens':response.usage.total_tokens,
    }
    print(response)
    print(content)
    print(token_dict)

    return response, content, token_dict

In [73]:
response, content, token_dict = get_completion_and_token_count("What is 1+1?")

ChatCompletion(id='chatcmpl-92alWLIj2RHWdi0LmTYY0NGzVA8aa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "result": 2\n}', role='assistant', function_call=None, tool_calls=None))], created=1710405010, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=9, prompt_tokens=18, total_tokens=27))
{
  "result": 2
}
{'prompt_tokens': 18, 'completion_tokens': 9, 'total_tokens': 27}


In [22]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
encoding.encode("What is 1+1?") # no. of input tokens counted by OpenAI API

[3923, 374, 220, 16, 10, 16, 30]

# ***Chat API : LangChain***

In [ ]:
# !pip install langchain

# Model

In [75]:
from langchain.chat_models import ChatOpenAI

In [117]:
import os
os.environ['OPENAI_API_KEY'] = openai_api_key # MUST
# To control the randomness and creativity of the generated text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)

# Prompt template - Input into the model

In [94]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [95]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [102]:
print(prompt_template.messages[0].prompt)
print(prompt_template.messages[0].prompt.input_variables)

input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
['style', 'text']


['style', 'text']

In [99]:
customer_email = """
Hi how are you doing
"""
customer_style = """Spanish \
in a calm and respectful tone
"""
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [106]:
print(type(customer_messages))
print(customer_messages)
print(type(customer_messages[0]))
print(customer_messages[0])

<class 'list'>
[HumanMessage(content='Translate the text that is delimited by triple backticks into a style that is Spanish in a calm and respectful tone\n. text: ```\nHi how are you doing\n```\n')]
<class 'langchain_core.messages.human.HumanMessage'>
content='Translate the text that is delimited by triple backticks into a style that is Spanish in a calm and respectful tone\n. text: ```\nHi how are you doing\n```\n'


In [118]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

In [119]:
print(customer_response.content)

Hola, ¿cómo estás?


# Parse the LLM output string into a Python dictionary

In [184]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [246]:
email_body_schema = ResponseSchema(name="email_body",
                                      description="Extract email body")

email_from_schema = ResponseSchema(name="email_sender",
                                      description="Extract sender email")

email_from_name_schema = ResponseSchema(name="email_sender_name",
                                      description="Extract sender name")

email_from_designation_schema = ResponseSchema(name="email_sender_designation",
                                      description="Extract sender designation")

email_from_organization_schema = ResponseSchema(name="email_sender_organization",
                                      description="Extract sender orgaization")

email_date_schema = ResponseSchema(name="email_date",
                                       description="Extract email date")

email_to_schema = ResponseSchema(name="email_recipients",
                                      description="Extract comma separated email recipients")

response_schemas = [email_from_schema,
                    email_from_name_schema,
                    email_from_designation_schema,
                    email_from_organization_schema,
                    email_to_schema,
                    email_date_schema,
                    email_body_schema]

In [247]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [248]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"email_sender": string  // Extract sender email
	"email_sender_name": string  // Extract sender name
	"email_sender_designation": string  // Extract sender designation
	"email_sender_organization": string  // Extract sender orgaization
	"email_recipients": string  // Extract comma separated email recipients
	"email_date": string  // Extract email date
	"email_body": string  // Extract email body
}
```


In [249]:
email = "From: Doe, John <JohnDoe@xyz.com> \
Sent: Wednesday, March 15, 2023 11:21 PM \
To: Hollester, Mike <@MHollester.com>; Parrera, Nicky <NParrera@xyz.com> \
Cc: Bush, Johnny <JBush@xyz.com>; Herding, Bill <BHerding@xyz.com> \
Subject: RE: Interested in AI product  \
I hope this email finds you well. \
I am reaching out to inquire about your company's AI offerings and services. \
Could you please provide more information regarding your capabilities and solutions in this field? \
Shawn Ponting \
Senior Product Manager \
Great Corporation"

In [250]:
review_template_2 = """\
For the following text, extract the following information:

email_body: Extract email body If this information is not found,output Unknown Email Body
email_sender: Extract sender email If this information is not found, output Unknown Sender.
email_sender_designation: Extract sender designation If this information is not found, output Unknown Sender Designation.
email_sender_organization: Extract sender organization If this information is not found, output Unknown Sender Organization.
email_date: Extract email date If this information is not found, output Unknown Date.
email_recipients: Extract comma separated email recipients If this information is not found, output Unknown Recipients.

text: {text}

{format_instructions}
"""

In [251]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=email,
                                format_instructions=format_instructions)

In [252]:
print(messages[0].content)

For the following text, extract the following information:

email_body: Extract email body If this information is not found,output Unknown Email Body 
email_sender: Extract sender email If this information is not found, output Unknown Sender.
email_sender_designation: Extract sender designation If this information is not found, output Unknown Sender Designation.
email_sender_organization: Extract sender organization If this information is not found, output Unknown Sender Organization.
email_date: Extract email date If this information is not found, output Unknown Date.
email_recipients: Extract comma separated email recipients If this information is not found, output Unknown Recipients.

text: From: Doe, John <JohnDoe@xyz.com> Sent: Wednesday, March 15, 2023 11:21 PM To: Hollester, Mike <@MHollester.com>; Parrera, Nicky <NParrera@xyz.com> Cc: Bush, Johnny <JBush@xyz.com>; Herding, Bill <BHerding@xyz.com> Subject: RE: Interested in AI product  I hope this email finds you well. I am reac

In [253]:
response = chat(messages)

In [254]:
print(response.content)

```json
{
	"email_sender": "JohnDoe@xyz.com",
	"email_sender_name": "Doe, John",
	"email_sender_designation": "Senior Product Manager",
	"email_sender_organization": "Great Corporation",
	"email_recipients": "Hollester, Mike <@MHollester.com>, Parrera, Nicky <NParrera@xyz.com>, Bush, Johnny <JBush@xyz.com>, Herding, Bill <BHerding@xyz.com>",
	"email_date": "Wednesday, March 15, 2023 11:21 PM",
	"email_body": "I hope this email finds you well. I am reaching out to inquire about your company's AI offerings and services. Could you please provide more information regarding your capabilities and solutions in this field?"
}
```


In [255]:
output_dict = output_parser.parse(response.content)

In [256]:
output_dict

{'email_sender': 'JohnDoe@xyz.com',
 'email_sender_name': 'Doe, John',
 'email_sender_designation': 'Senior Product Manager',
 'email_sender_organization': 'Great Corporation',
 'email_recipients': 'Hollester, Mike <@MHollester.com>, Parrera, Nicky <NParrera@xyz.com>, Bush, Johnny <JBush@xyz.com>, Herding, Bill <BHerding@xyz.com>',
 'email_date': 'Wednesday, March 15, 2023 11:21 PM',
 'email_body': "I hope this email finds you well. I am reaching out to inquire about your company's AI offerings and services. Could you please provide more information regarding your capabilities and solutions in this field?"}